In [ ]:
####set up environment

import pymssql
import pandas as pd
conn = pymssql.connect(server='#########', user='#########', password='#########', database='############')

cursor = conn.cursor()

cursor.execute('SELECT REMARKS,EndItemId,Id FROM Da2408_13_1')

row = cursor.fetchone()

while row:

  print(str(row[0]) + " " + str(row[1]))

  row = cursor.fetchone()


q='''SELECT REMARKS,Id,EndItemId, CorrectiveAction, DiscoveryDateTime, CorrectiveDateTime,CorrectiveWorkUnitCode FROM Da2408_13_1'''
df1=pd.read_sql(q,conn)
df1Sample=df1.sample(20000)
df1Sample

In [ ]:
 	REMARKS 	Id 	EndItemId 	CorrectiveAction 	DiscoveryDateTime 	CorrectiveDateTime 	CorrectiveWorkUnitCode
298088 	OPERATE IAW THE LIMITATIONS AND RESTRICTIONS S... 	3cdecad9-5800-4d32-98a8-402c2be39307 	25c4f83e-e60a-5e13-894f-79695829aea1 	UPDATED AWR BULK ENTRY IAW DA PAM 738-751. 	2021-03-27 08:46:45.897 	2021-03-31 00:36:00 	02
143208 	W415 INSP DUE 	909142a6-0628-4a9e-80b2-1fd3faecae6d 	4e3e705c-32ff-4dbe-8673-4063df539e8d 	COMPLETED IAW TM 1-1520 LONGBOW/APACHE 	2021-04-14 12:24:30.570 	2021-04-16 09:15:00 	35A
218767 	R 40 PANEL UPPER ORINGS DETERIORATED 	df1a5134-0ef7-589c-a37e-94f4137dbbc1 	bf1c0c9a-ae7c-5828-8d59-5ff20d7e225a 	REPLACED ORINGS IAW TM 1-1520-LONGBOW/APACHE 	2019-02-04 19:03:00.000 	2019-02-15 15:24:00 	02
475536 	FMC MOC DUE FOR COMPLETION OF CCI/WASH\r\n 	091f89c1-ac55-4c96-ac52-9a09aee2fa12 	aec1adb4-651b-58cb-acf0-cee9213ff3e3 	COMPLETED IAW 1-1520-LONGBOW/APACHE 	2020-09-27 07:44:05.123 	2020-09-30 03:47:00 	00
526159 	DURING MSPU SURVEY NO.4 DRIVE SHAFT CI YELLOW ... 	9e7c38eb-1ab3-45df-b5c9-f08fff985ef8 	e99d5551-aba4-5ab9-9948-dedd0a6997f3 	None 	2021-05-19 14:36:26.750 	NaT 	None
... 	... 	... 	... 	... 	... 	... 	...
216575 	NIU CHECK DUE FOR COMPLETION OF PMS 	989dec47-d4e2-50cf-a1fd-23d522b566f4 	bf1c0c9a-ae7c-5828-8d59-5ff20d7e225a 	MOC COMPLETE OK IAW TM 1-1520-LONGBOW/APACHE 	2019-08-16 01:09:00.000 	2019-08-16 02:40:00 	10P
703 	PMD Due 	eb3883df-50ce-4e8e-a8c8-20423b798a28 	28fe7469-b0ed-5162-9220-0008631cf16a 	COMPLETED 	2019-11-20 01:12:00.000 	2019-11-20 01:16:00 	00
242421 	24 hour cure time due (or untill cured) for pr... 	7a3df444-32a7-440b-886a-8b13a37ad9d5 	4920da1d-e6a3-57e9-9e3d-68f8fdde2c1c 	sealant is cured iaw tm 1-1520-longbow/apache 	2020-05-06 13:53:57.477 	2020-05-07 06:17:00 	02
382678 	INSP A800 FOLLOWING THE LAST FLIGHT OF THE DAY... 	543e0c04-2e0f-45aa-ade3-418bc2d57113 	2c6d9a5d-f5e5-5c92-a049-b30bd5856a22 	comp iaw awr 2014E-A48 R7 icw pmd dated 29nov2019 	2019-11-25 22:48:00.000 	2019-11-29 22:31:00 	02
87026 	INSP A867 - INSPECT THE MR FEATHER BEARING PRI... 	fcd13395-243f-5317-ba51-884e6ab3c608 	d09d7a51-5cfb-581a-ade7-27a4948e91ed 	COMP IAW PART 2 OF ADDENDUM 1 PER H-64-19-ASA... 	2019-07-18 19:45:00.000 	2019-07-19 00:52:00

In [ ]:
#####Tokenize text field from data
import nltk
nltk.download('punkt')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
tagged_data = [TaggedDocument(words=word_tokenize(term_def.lower()), tags=[list_id[i]]) for i, term_def in enumerate(list_docs)]

In [ ]:
#look at some example terms
model.wv.most_similar("engine")
model.wv.most_similar("hazard")
model.wv.most_similar("emergency")

In [ ]:
###### Build Doc2Vec model using tokenized text field 
max_epochs = 50
model = Doc2Vec()  # of course, if non-default parameters needed, use them here

                   # but most users won't need to change alpha/min_alpha at all

model.build_vocab(tagged_data)
model.train(tagged_data[1:10000], total_examples=model.corpus_count, epochs=max_epochs)
model.save("d2vSimple.model")

In [ ]:
#create 2 dimensional space from components 

tsne = TSNE(n_components=2)

X_tsne = tsne.fit_transform(X)

df = pd.DataFrame(X_tsne, index=doc_tags[1:10000], columns=['x', 'y'])

In [ ]:
###parser function used to generate columns for chart
def parser(input_file):

    df_dx = pd.DataFrame(columns=['id', 'EndItemId', 'Remark'])

 

    with open(input_file, "r") as f:

        term_id = term_name = term_def = None

        term_is_a = []

 

        for line in f:

            line = line.rstrip('\n')

 

            if "[Term]" in line or "[Typedef]" in line:            

                if term_def:

                    df_dx.loc[len(df_dx)] = [term_id, term_name, term_def, term_is_a]

 

                term_id = term_name = term_def = None

                term_is_a = []

            elif "id: " == line[0:4]:

                term_id = line.split("id: ")[1]

            elif "name: " in line:

                term_name = line.split("name: ")[1]

            elif "def: " in line:

                temp = line.split("def: ")[1]

                term_def = temp.split("\"")[1]

            elif "is_a: " in line:

                temp = line.split("is_a: ")[1]

                temp = temp.split(" ! ")[0]

                term_is_a.append(temp)

 

    return df_dx

In [ ]:
###plot Cluster results, this was done in a Tableau Server
def plotScatter(keyword):

    fig = plt.figure(figsize=(10,15))

    ax = fig.add_subplot(1, 1, 1)

 

    pos_found_x = []

    pos_found_y = []

    found_names = []

 

    pos_rest_x = []

    pos_rest_y = []

 

    for term_id, pos in df.iterrows():

        term_name = df_dx[df_dx['id'] == term_id]['EndID'].values[0]

        term_def = df_dx[df_dx['id'] == term_id]['def'].values[0].lower()

 

        if keyword in term_name:

            pos_found_x.append(pos['x'])

            pos_found_y.append(pos['y'])

        elif keyword in term_def:

            pos_found_x.append(pos['x'])

            pos_found_y.append(pos['y'])

        else:

            found = False

            is_a_list = df_dx[df_dx['id'] == term_id]['is_a'].tolist()[0]

 

            for is_a_id in is_a_list:

                if len(df_dx[df_dx['id'] == is_a_id]) > 0:

                    if keyword in df_dx[df_dx['id'] == is_a_id]['name'].values[0]:

                        pos_found_x.append(pos['x'])

                        pos_found_y.append(pos['y'])

                        found = True

                        break

                    elif keyword in df_dx[df_dx['id'] == is_a_id]['def'].values[0].lower():

                        pos_found_x.append(pos['x'])

                        pos_found_y.append(pos['y'])

                        found = True

                        break

 

            if found == False:

                pos_rest_x.append(pos['x'])

                pos_rest_y.append(pos['y']) 

 

    ax.scatter(pos_rest_x, pos_rest_y, c='blue')       

    ax.scatter(pos_found_x, pos_found_y, c='red')